In [20]:
import os
import open3d as o3d
import numpy as np
import plotly
import json
import sklearn

In [2]:
# base_dir = '../RES/point_cloud_train/clouds_stereo'
base_dir = '../RES/point_cloud_gt/clouds_tof'

In [3]:
for file_name in os.listdir(base_dir):
    file_path = os.path.join(base_dir, file_name)
    cmd = f'unzip {file_path} -d {base_dir}'
    print(cmd)
    os.system(cmd)

unzip ../RES/point_cloud_gt/clouds_tof/cloud_0_0074.pcd -d ../RES/point_cloud_gt/clouds_tof
unzip ../RES/point_cloud_gt/clouds_tof/cloud_0_0132.pcd.zip -d ../RES/point_cloud_gt/clouds_tof
unzip ../RES/point_cloud_gt/clouds_tof/cloud_0_0080.pcd -d ../RES/point_cloud_gt/clouds_tof
unzip ../RES/point_cloud_gt/clouds_tof/cloud_0_0013.pcd.zip -d ../RES/point_cloud_gt/clouds_tof
unzip ../RES/point_cloud_gt/clouds_tof/cloud_0_0076.pcd.zip -d ../RES/point_cloud_gt/clouds_tof
unzip ../RES/point_cloud_gt/clouds_tof/cloud_0_0061.pcd.zip -d ../RES/point_cloud_gt/clouds_tof
unzip ../RES/point_cloud_gt/clouds_tof/cloud_0_0132.pcd -d ../RES/point_cloud_gt/clouds_tof
unzip ../RES/point_cloud_gt/clouds_tof/cloud_0_0065.pcd.zip -d ../RES/point_cloud_gt/clouds_tof
unzip ../RES/point_cloud_gt/clouds_tof/cloud_0_0150.pcd -d ../RES/point_cloud_gt/clouds_tof
unzip ../RES/point_cloud_gt/clouds_tof/cloud_0_0010.pcd -d ../RES/point_cloud_gt/clouds_tof
unzip ../RES/point_cloud_gt/clouds_tof/cloud_0_0138.pcd -d .

In [4]:
for file_name in os.listdir(base_dir):
    if file_name.split('.')[-1] != 'pcd':
        continue
    
    file_path = os.path.join(base_dir, file_name)
    print(file_path)

    points = []
    colors = []
    with open(file_path, 'r') as inf:
        lines = inf.readlines()
        for line in lines[11:]:
            values = line.strip().split(' ')
            point = [float(i) for i in values[:3]]
            color = int(values[3])
#             color = [color >> 16, (color >> 8) & 2**8, color & 2**8]
            color = [(color // 2**24) & 255, (color // 2**16) & 255, (color // 2**8) & 255, color & 255]
            points.append(point)
            colors.append(color)
            
    points_arr = np.array(points)
    colors_arr = np.array(colors, dtype=np.float) / 255
            
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_arr)
    pcd.colors = o3d.utility.Vector3dVector(colors_arr[:,:3])
    
    pcd = pcd.voxel_down_sample(voxel_size=0.05)

    o3d.visualization.draw_geometries([pcd])
    
#     viewer = o3d.visualization.Visualizer()
#     viewer.create_window()
#     viewer.add_geometry(pcd)
#     opt = viewer.get_render_option()
#     opt.show_coordinate_frame = True
#     opt.background_color = np.asarray([0.5, 0.5, 0.5])
#     viewer.run()
#     viewer.destroy_window()
        
#     break

../RES/point_cloud_gt/clouds_tof/cloud_0_0074.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0080.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0132.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0150.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0010.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0138.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0065.pcd


KeyboardInterrupt: 

In [6]:
for file_name in os.listdir(base_dir):
    if file_name.split('.')[-1] != 'pcd':
        continue
        
    out_file_name = str(int(file_name.split('.')[0].split('_')[2]))
    
    file_path = os.path.join(base_dir, file_name)
    print(file_path)

    os.system(f'../scene_analyzer.py -i {file_path} -o jsons/{out_file_name}.json')

../RES/point_cloud_gt/clouds_tof/cloud_0_0074.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0080.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0132.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0150.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0010.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0138.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0065.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0030.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0047.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0054.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0006.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0013.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0061.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0003.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0020.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0068.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0071.pcd
../RES/point_cloud_gt/clouds_tof/cloud_0_0076.pcd


# Compute auc rog

In [14]:
our_results = {}
for file_name in os.listdir('jsons'):
    file_idx = int(file_name.split('.')[0])
    
    with open(f'jsons/{file_name}', 'r') as inf:
        data = json.load(inf)
    
    dop = data['door_open_percent']
    our_results[file_idx] = dop
    
our_results

{68: 8,
 30: 8,
 6: 8,
 71: 16,
 138: 100,
 61: 33,
 20: 8,
 54: 16,
 13: 0,
 80: 25,
 132: 25,
 76: 33,
 65: 0,
 74: 0,
 150: 0,
 10: 0,
 47: 8,
 3: -1}

In [19]:
trues_str = '''0003  UNKNOWN
0006  CLOSED
0010  CLOSED
0013  SEMI
0020  CLOSED
0030  SEMI
0047  CLOSED
0054  SEMI
0061  SEMI
0065  CLOSED
0068  SEMI
0071  SEMI
0074  CLOSED
0076  CLOSED
0080  SEMI
0132  OPEN
0138  SEMI
0150  CLOSED'''

trues_dict = {}
for line in trues_str.split('\n'):
    idx, status = line.split('  ')
    idx = int(idx)
    is_closed = status == 'CLOSED'
    trues_dict[idx] = is_closed
print(idx, trues_dict)

150 {3: False, 6: True, 10: True, 13: False, 20: True, 30: False, 47: True, 54: False, 61: False, 65: True, 68: False, 71: False, 74: True, 76: True, 80: False, 132: False, 138: False, 150: True}


In [24]:
keys_list = []
trues_list = []
preds_list = []
for k,v in trues_dict.items():
    if k == 3:
        continue
    keys_list.append(k)
    trues_list.append(v)
    preds_list.append((100-our_results[k])/100)
    
print(keys_list)
print(trues_list)
print(preds_list)

[6, 10, 13, 20, 30, 47, 54, 61, 65, 68, 71, 74, 76, 80, 132, 138, 150]
[True, True, False, True, False, True, False, False, True, False, False, True, True, False, False, False, True]
[0.92, 1.0, 1.0, 0.92, 0.92, 0.92, 0.84, 0.67, 1.0, 0.92, 0.84, 1.0, 0.67, 0.75, 0.75, 0.0, 1.0]


In [23]:
sklearn.metrics.roc_auc_score(trues_list, preds_list)

0.7847222222222222